In [20]:
import getpass
import os

from langchain_openai import AzureChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.output_parsers import StrOutputParser


# os.environ["AZURE_OPENAI_API_KEY"] = getpass.getpass()

#os.environ["AZURE_OPENAI_API_VERSION"] = "2024-10-01-preview"

# Instantiate the model with environment variables
model = AzureChatOpenAI(
    azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],
    api_key=os.environ["AZURE_OPENAI_API_KEY"],
    openai_api_version=os.environ["AZURE_OPENAI_API_VERSION"]
)

In [36]:
# import os
# print("AZURE_OPENAI_ENDPOINT:", os.environ.get("AZURE_OPENAI_ENDPOINT"))
# print("AZURE_OPENAI_API_KEY:", os.environ.get("AZURE_OPENAI_API_KEY"))
# print("AZURE_OPENAI_API_VERSION:", os.environ.get("AZURE_OPENAI_API_VERSION"))


In [23]:
messages = [
    SystemMessage(content="Translate the following from English into Italian"),
    HumanMessage(content="hi!"),
]

model.invoke(messages)

AIMessage(content='Ciao!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 3, 'prompt_tokens': 20, 'total_tokens': 23, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'gpt-4o-mini', 'system_fingerprint': 'fp_d54531d9eb', 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {}}], 'finish_reason': 'stop', 'logprobs': None, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'protected_material_code': {'filtered': False, 'detected': False}, 'protected_material_text': {'filtered': False, 'detected': False}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}, id='run-367ec28b-cf6a-4d6e-80fd-4df028ee9435-0', usage_metadata={'input_tokens': 20, 'output_tokens': 3, 'total_tokens': 23, 'input_token_details': {}, 'output_token_details': {}})

In [24]:
# print(model)
# print(messages)

parser = StrOutputParser()

model.invoke(messages)

# print(result.content)

AIMessage(content='Ciao!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 3, 'prompt_tokens': 20, 'total_tokens': 23, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'gpt-4o-mini', 'system_fingerprint': 'fp_d54531d9eb', 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {}}], 'finish_reason': 'stop', 'logprobs': None, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'protected_material_code': {'filtered': False, 'detected': False}, 'protected_material_text': {'filtered': False, 'detected': False}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}, id='run-5558f275-2909-43b2-9536-bbcbfc4206e8-0', usage_metadata={'input_tokens': 20, 'output_tokens': 3, 'total_tokens': 23, 'input_token_details': {}, 'output_token_details': {}})

In [25]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

In [26]:
result = model.invoke(messages)
parser.invoke(result)

'Ciao!'

In [27]:
# print(parser.invoke(result))
chain = model | parser
print(chain.invoke(messages))

Ciao!


In [28]:
from langchain_core.prompts import ChatPromptTemplate

In [29]:
system_template = "Translate the following into {language}:"

In [30]:
prompt_template = ChatPromptTemplate.from_messages(
    [("system", system_template), ("user", "{text}")]
)

In [31]:
result = prompt_template.invoke({"language": "italian", "text": "hi"})

result

ChatPromptValue(messages=[SystemMessage(content='Translate the following into italian:', additional_kwargs={}, response_metadata={}), HumanMessage(content='hi', additional_kwargs={}, response_metadata={})])

In [32]:
result.to_messages()

[SystemMessage(content='Translate the following into italian:', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='hi', additional_kwargs={}, response_metadata={})]

#### Chaining together components with LCEL

In [33]:
chain = prompt_template | model | parser

In [34]:
chain.invoke({"language": "italian", "text": "hi"})

'Ciao'

In [38]:
from langserve import RemoteRunnable

remote_chain = RemoteRunnable("http://localhost:8000/chain/")
remote_chain.invoke({"language": "italian", "text": "hi"})

'Ciao'